# Feature Enginering Notebook for Dengue Transfer Learning Project

### Dataset
Dengue ML datasets track environmental and temporal factors influencing Aedes mosquito breeding and virus transmission in tropical regions like San Juan and Iquitos.

- #### Temporal Features
    - **city**: Location identifier (e.g., 'sj' for San Juan, 'iq' for Iquitos)—captures city-specific mosquito/dengue patterns.
    - **year, weekofyear, week_start_date**: Time granularity for seasonality; dengue peaks during rainy seasons (weekofyear critical for lagged effects).

- #### Vegetation Indices (NDVI)
    - **ndvi_ne, ndvi_nw, ndvi_se, ndvi_sw**: Normalized Difference Vegetation Index by city quadrant. Higher NDVI indicates lush vegetation providing mosquito shade/breeding sites; key for Aedes habitat detection via satellite.

- #### Precipitation \& Water
    - **precipitation_amt_mm**: Rainfall amount—creates standing water breeding sites.
    - **reanalysis_precip_amt_kg_per_m2, reanalysis_sat_precip_amt_mm**: Reanalysis (modeled) precipitation variants confirming observed rain.
    - **station_precip_mm**: Ground station measurements—most direct rain proxy.

- #### Temperature Metrics
    - **reanalysis_air_temp_k, reanalysis_avg_temp_k, reanalysis_max_air_temp_k, reanalysis_min_air_temp_k**: Reanalysis temps in Kelvin; optimal Aedes range 26-32°C accelerates larval development/virus replication.
    - **station_avg_temp_c, station_max_temp_c, station_min_temp_c**: Station temps in Celsius—ground truth validation.
    - **station_diur_temp_rng_c**: Diurnal range; wider swings stress mosquitoes.
    - **reanalysis_tdtr_k**: Temperature diurnal temperature range (reanalysis).

- #### Humidity \& Moisture
    - **reanalysis_dew_point_temp_k**: Dew point—direct humidity proxy; high values (>20°C) favor mosquito survival.
    - **reanalysis_relative_humidity_percent**: Relative humidity %—critical for egg/larval viability.
    - **reanalysis_specific_humidity_g_per_kg**: Absolute moisture content.


### For a fair fight between LightGBM and our LSTM, both models will have the same starting lineup of features. These include seasonal cues, vegetation signals, missing data hints, and early low-case periods—everything else is just how each model likes to learn over time.

### Notebook sections for the third project notebook (Feature Enginering)
1. Get Data
2. Feature Engineering
    - Common for both models
- Intermittent multicolinearity assessment and removal feature list building for Feature Selection

In [1]:
import sys
import os
from pathlib import Path
from typing import List, Tuple, Any, Dict, Optional
import gc
import itertools
import logging
logging.basicConfig(level=logging.INFO)

# Set one level up as project root|
if os.path.abspath("..") not in sys.path:
    sys.path.insert(0, os.path.abspath(".."))
    
from src.config import ProjectConfig  # project config file parser
from src.utils.eda import value_streaks, top_correlations
from src.utils.visualizations import (compute_correlations_matrix,
                display_distributions, random_color, random_colormap,
                display_timeseries)

import pandas as pd
import numpy as np
import random
import time
from datetime import timedelta

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from src.utils.eda import top_correlations, top_vif
from src.utils.utils import _check_feature_presence, load_file, save_file
from src.preprocessing.clean import (drop_nan_rows, cap_outliers,
                                    median_groupwise_impute, pipe_clean)

from src.preprocessing.engineer.base import (reduce_features,
                                            add_missingness_features,
                                            low_value_targets)

from src.preprocessing.engineer.temporal import (circular_time_features,
                                                dynamic_temporal_features)

from src.preprocessing.engineer.pipeline import pipe_engineer

from src.preprocessing.select import remove_features

from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
# from matplotlib.axis import Axis
# from matplotlib.dates import MonthLocator, YearLocator, DateFormatter
import seaborn as sns

In [2]:
cnfg = ProjectConfig.load_configuration()
PATH_TO_RAW_DATA = cnfg.data.dirs["raw"]
PATH_TO_INTERMEDIATE_DATA = cnfg.data.dirs["intermediate"]

FILE_TRAIN_RAW= cnfg.data.files["features_train"]
FILE_TEST_RAW = cnfg.data.files["features_test"]
FILE_LABELS_RAW = cnfg.data.files["labels_train"]

PATH_TO_INTERMEDIATE_DATA = cnfg.data.dirs["intermediate"]
FILE_NAN_CLEAN = cnfg.data.files["nan_mask"]
FILE_TRAIN_CLEAN = cnfg.data.files["features_clean"]
FILE_LABELS_CLEAN = cnfg.data.files["labels_clean"]

TARGET = cnfg.preprocess.feature_groups["target"]
ENV_FEAT_PREFIX = cnfg.preprocess.feature_groups["env_prefixes"]
CITYGROUP_FEAT = cnfg.preprocess.feature_groups["city"]
WEEK_FEAT = cnfg.preprocess.feature_groups["week"]
DATETIME_FEAT = cnfg.preprocess.feature_groups["datetime"]

## Get Data

- Raw data

In [3]:
df_train_raw = load_file(path=PATH_TO_RAW_DATA / FILE_TRAIN_RAW, datetime_col=DATETIME_FEAT)
df_test_raw = load_file(path=PATH_TO_RAW_DATA / FILE_TEST_RAW, datetime_col=DATETIME_FEAT)
df_labels_raw = load_file(path=PATH_TO_RAW_DATA / FILE_LABELS_RAW)
list_raw_df = [df_train_raw, df_test_raw, df_labels_raw]
env_features = [f for f in df_train_raw if f.startswith(tuple(ENV_FEAT_PREFIX))]

- Cleaned data

In [4]:
# %%time
# # use pipeline output
# cleaned_data = pipe_clean(overwrite_files=True)
# df_train_clean = cleaned_data["X_clean_data"]
# df_labels_clean = cleaned_data["y_clean_data"]
# df_nan_mask = cleaned_data["nan_mask_data"]

In [5]:
%%time
# or just load from disc:
df_train_clean = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_TRAIN_CLEAN, datetime_col=DATETIME_FEAT)
df_labels_clean = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_LABELS_CLEAN)
df_nan_mask = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_NAN_CLEAN)

CPU times: user 43.7 ms, sys: 1.05 ms, total: 44.7 ms
Wall time: 44.4 ms


## Common engineering baseline
- [X] initial VIF based multicolinearity feature drop
- [X] Missingness features
- [X] combine north and south NDVIs
- [X] Process zero/low value target value streaks (add `iq_initial_low_case_streak` feature)
- [X] Cyclical time features
- [X] First VIF based multicolinearity feature drop - see if any engineered features create new multicolinearities
- [X] Lags / rolling stats
- [X] Second VIF - see if Lags / rolling stats create new multicolinearities

### Remove selected multicolinear features (Common Feature Selection baseline)
- This is Development specific arrangement - in production hard-coded feature removal will be after engineeringn in feature selection pipeline.
- [X] Remove initial `config.yaml` milticolinear features form dataframe
    - Initial assesment of pairwise correlations in EDA 
- [X] Assess city-wise VIF to EDA instead of correlation matrix for one-vs-all relationships.
- [X] Remove features with VIF > 10
- [X] always prefer station_* over reanalysis_*

In [6]:
print("Pre-feature removal Variance Inflation Factor ratios: VIF 0.")
temp_vif_pre = df_train_clean.groupby(by=CITYGROUP_FEAT).apply(lambda group: top_vif(data=group), include_groups=False)
temp_vif_pre =  temp_vif_pre.loc["iq"].join(temp_vif_pre.loc["sj"], lsuffix="_iq", rsuffix="_sj", how="inner")
temp_vif_pre["vif_total"] = top_vif(data=df_train_clean).values
temp_vif_pre.sort_values(by="vif_total", ascending=False, na_position="first")

Pre-feature removal Variance Inflation Factor ratios: VIF 0.


/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,vif_iq,vif_sj,vif_total
reanalysis_sat_precip_amt_mm,inf,inf,inf
precipitation_amt_mm,inf,inf,inf
reanalysis_dew_point_temp_k,455.961293,1704.099235,462.995723
reanalysis_specific_humidity_g_per_kg,420.700001,526.632565,380.575453
reanalysis_relative_humidity_percent,113.185458,305.963736,209.573186
reanalysis_air_temp_k,76.656998,1139.740207,182.109533
reanalysis_avg_temp_k,37.506133,276.791687,66.694328
reanalysis_tdtr_k,16.028064,3.430572,45.410972
reanalysis_max_air_temp_k,5.805232,17.219178,26.448369
reanalysis_min_air_temp_k,5.520049,19.240000,23.232690


In [7]:
cnfg.preprocess.multicolinear["removal_list"]

['reanalysis_sat_precip_amt_mm',
 'reanalysis_dew_point_temp_k',
 'reanalysis_air_temp_k',
 'reanalysis_specific_humidity_g_per_kg',
 'reanalysis_avg_temp_k',
 'reanalysis_max_air_temp_k',
 'reanalysis_min_air_temp_k']

In [8]:
df_train_clean = remove_features(X=df_train_clean)

In [9]:
corr_threshold=0.8
print(f"City-stratified correlations exceeding {corr_threshold}:")
df_train_clean.groupby(
    by=CITYGROUP_FEAT).apply(
        lambda group: top_correlations(
            data=group, 
            corr_threshold=corr_threshold
        ), include_groups=False)

City-stratified correlations exceeding 0.8:


city                                                          
iq    ndvi_ne             ndvi_sw                                 0.840730
      reanalysis_tdtr_k   reanalysis_relative_humidity_percent   -0.900588
sj    station_min_temp_c  station_avg_temp_c                      0.898016
      station_max_temp_c  station_avg_temp_c                      0.864780
      ndvi_sw             ndvi_se                                 0.819673
dtype: float64

In [10]:
print("Post-initial-feature removal Variance Inflation Factor ratios: VIF 1.")
temp_vif_pre = df_train_clean.groupby(by=CITYGROUP_FEAT).apply(lambda group: top_vif(data=group), include_groups=False)
temp_vif_pre =  temp_vif_pre.loc["iq"].join(temp_vif_pre.loc["sj"], lsuffix="_iq", rsuffix="_sj", how="inner")
temp_vif_pre["vif_total"] = top_vif(data=df_train_clean).values
temp_vif_pre.sort_values(by="vif_total", ascending=False, na_position="first")

Post-initial-feature removal Variance Inflation Factor ratios: VIF 1.


,vif_iq,vif_sj,vif_total
reanalysis_relative_humidity_percent,6.139555,3.111873,9.211951
reanalysis_tdtr_k,5.927034,1.819038,7.742019
ndvi_ne,4.748647,1.676724,6.738472
ndvi_sw,4.025909,3.234273,6.650551
station_diur_temp_rng_c,3.016941,3.041026,5.982186
station_avg_temp_c,2.903773,15.205912,4.242656
ndvi_nw,2.845429,1.950984,4.045501
ndvi_se,2.690311,3.280663,3.942102
station_max_temp_c,2.525159,7.615379,3.264081
station_min_temp_c,2.246606,9.490197,2.711820


**Conclusion**:
- Initial VIF risky - perfect correlations (`inf`), multiple features score well above 100
- After selected feature removal `vif_total` Variance Inflation Factor combined  and separate for both cities is below 10:
  - Among highest VIF scoring overall remains `reanalysis_relative_humidity_percent`, but it is important as there are no alternative station humiidity data and humidity is important for dengue detection domain. 
- `San Juan` still suffers from higher Multicolinearity that reflects geographical characteristic (more stable climate):
    - Top VIF scores for `San Juan` reach 15.2 and are in crucial station  temperature measurements. `station_avg_temp_c` could be potentially removed, but risks hiding important signals in `Iquitos`.
    - Keep `station_avg_temp_c` and observe model results:
        - VIF score of ~ 15-20 should be well handled by `LightGBM`
        - `LTSM` should handle colinearity even better. If `LTSM` `San Juan` -> `Iquitos` transfer learn underperforms, attempt adding `station_avg_temp_c` to `config.yaml` feature removal list. 
- VIF factors for `Iquitos` are all acceptable in range below ~6.1.
- Remaining high correlation pairs are differenft for both cities confirming that closeness of these features are city specific and thus may encompass important signals for models. So they are kept.

### Add data misingness features
- Add missigness features that flag NaN rows for top NaN columns that have NaN rate above 1 %:
    - Idea - models, especially `LSTM` may learn from informatioun that "this datapoint was missing"
- [X] grouped `ndvi_n_missing` and `ndvi_s_missing` (`ndvi_n` and `ndvi_s` will be grouped downstream)
    - if feature space bloats - combine in single `ndvi_missing`
- [X] `station_missing` - group all station data missing flags ("station", "precipitation" prefixes)
- [X] `missing_pct_env_feat`:
    - normalized aggregation of all missing feature count/NaNs at a timestamp relative to a fixed set of core environmental variables, signals overall source data uncertainty.
    -  if feature space bloats - drop entirely
- [ ] (Optional) During SJ pretraining: Slightly stronger dropout on missingness features than on base features to hide city-correlated missingness

In [11]:
# # TODO remove after cleaning

# def add_missingness_features(X: pd.DataFrame,
#                              nan_mask: pd.DataFrame,
#                              aggregated_feat_name: Optional[str]=None,
#                              input_env_feat_prefixes: Optional[List[str]]=None,
#                              input_group_prefix: Optional[list]=None,
#                              output_feature_prefix: Optional[str]=None
#                             ) -> pd.DataFrame:
#     """
#     Add missingness indicator features to DataFrame using column prefix patterns.
#     1. (optional) Aggregated ratio of missing values across environment prefix columns.
#     2. Max missingness indicator (0/1) per feature group defined by prefixes
#     Fall back to config values if parameters unspecified.
#     :param X: Input DataFrame to add missingness features to.
#     :param nan_mask: Boolean DataFrame same shape as X where True indicates missing.
#     :param aggregated_feat_name: Name for aggregated missingness ratio feature. 
#                                 Uses config default if None.
#     :param input_env_feat_prefixes: List of prefixes to match environment columns 
#                                    for aggregated ratio. Uses config if None.
#     :param input_group_prefix: List of prefix lists or single strings defining 
#                               feature groups for max missingness indicators. 
#                               Mixed format supported: `[["station", "precip"], "ndvi_s"]`.
#                               Uses config if None.
#     :param output_feature_prefix: Prefix for new group missingness columns 
#                                  (e.g., "missing_station_max"). Uses config if None.
#     :return: X with additional missingness feature column(s).
#     :raises ValueError: If no columns match environment prefixes.
#     """
#     X_missing = X.copy()
#     config = cnfg.preprocess.missingness_features
#     aggregated_feat_name = aggregated_feat_name or config.get("aggregated_feat_n")
#     input_env_feat_prefixes = input_env_feat_prefixes or cnfg.preprocess.feature_groups["env_prefixes"]
#     input_group_prefix = input_group_prefix or config["group_prefixes"]
#     output_feature_prefix = output_feature_prefix or config["new_feature_prefix"]

#     if aggregated_feat_name and input_env_feat_prefixes:
#         env_cols = nan_mask.columns[nan_mask.columns.str.startswith(tuple(input_env_feat_prefixes))]
#         denominator = len(env_cols)
#         if denominator == 0:
#             raise ValueError("No columns match environment prefix/es '{input_env_feat_prefixes}'.")
#         X_missing[f"{output_feature_prefix}{aggregated_feat_name}"
#             ] = nan_mask[env_cols].sum(axis=1) / denominator

#     for prefix in input_group_prefix:
#         if isinstance(prefix, str):
#             prefix = [prefix]
#         features = nan_mask.columns[
#             nan_mask.columns.str.startswith(tuple(prefix))]
#         if len(features) == 0:
#             logging.warning(f"No columns match prefix/es '{prefix}' - skipping.")
#             continue
#         feature_name = f"{output_feature_prefix}{prefix[0]}"
#         X_missing[feature_name] = nan_mask[features].agg("max", axis=1)
    
#     return X_missing

In [12]:
df_train_eng = add_missingness_features(X=df_train_clean, nan_mask=df_nan_mask)
df_train_eng.columns[-4:]

Index(['missing_pct_env_feat', 'missing_station', 'missing_ndvi_s',
       'missing_ndvi_n'],
      dtype='object')

### Combine North and South NDVIs
- [X] combine primary veatures (method `mean`)
- [X] combine related misingness flags (method `max`) - done upstream in `add_missingness_features()`

In [13]:
# # TODO remove after cleaning

# def reduce_features(X: pd.DataFrame, 
#                     input_feat_groups: List[List[str]]=None,
#                     output_feat_names: List[str]=None,
#                    function: str=None):
#     """
#     Aggregate multiple feature groups into single reduced features using specified function.
#     Combine input features and drop originals.
    
#     :param X: Input pandas DataFrame.
#     :param input_feat_groups: List of feature group lists to aggregate. Default None uses 
#            config.yaml settings (e.g., [['ndvi_ne', 'ndvi_nw']]).
#     :param output_feat_names: Output column names for aggregated features. Default None uses 
#            config.yaml settings (e.g., ['ndvi_north']).
#     :param function: Aggregation function string ('mean', 'sum', 'median'). Default None uses 
#            config.yaml settings (e.g 'mean').
#     :return: DataFrame with reduced features. Original input columns dropped.
#     """
#     X_reduced = X.copy()
#     if input_feat_groups is None:
#         input_feat_groups = cnfg.preprocess.combine_features["input_groups"]
#     if output_feat_names is None:
#         output_feat_names = cnfg.preprocess.combine_features["output_names"]
#     if function is None:
#         function = cnfg.preprocess.combine_features["aggregation"]

        
#     if not len(input_feat_groups) == len(output_feat_names):
#         raise ValueError(f"Input feature groups {input_feat_groups} mismatch target keys {output_feat_names}")
#     missing_features = set(itertools.chain(*input_feat_groups)) - set(X.columns)
#     if missing_features:
#         raise ValueError(f"No {missing_features} features in input dataframe columns: {X.columns}")

#     for name, group in zip(output_feat_names, input_feat_groups):
#         X_reduced[name] = X_reduced[group].agg(function, axis=1)
#         X_reduced.drop(columns=group, inplace=True)
        
#     return X_reduced

In [14]:
df_train_eng = reduce_features(df_train_eng)
[f for f in df_train_eng.columns if f.startswith("ndvi")]

['ndvi_north', 'ndvi_south']

### Process zero/low value target value streaks
- [X] Feature engineer continuous low case streak counts for entire period's low values (ie `low_case_streak`)
- [X] Feature engineer binary low case streak marker for early low values (ie `initial_low_case_streak`)
- [X] avoid target leakage (do not use future data in calculating streaks - i.e. `zero_one_targets` function used for EDA)
- [ ] compare performance with and without `low_case_streak` and `initial_low_case_streak` features:
    - [ ] especially for `initial_low_case_streak`:
      - it has no transfer learning potential (ie not present in `San Juan`)
      -  all 0 in `San Juan` risk unstable model coeficienst
      -  75-week artifact won't repeat, potentialbecomes dead weight
      -  t-1 and t-4 case lags may capture low case persistence
    - [ ] run walk-forward CV i feature selection on IQ subset:
        - [ ] Check if improve MAPE/RMSE vs baseline?
        - [ ] If >3-5% lift, Iquitos underreporting hypothesis proven and has signal valuable for model

In [15]:
low_val_feats = cnfg.preprocess.low_value_streak_features
low_val_feats

{'target_streak_len_threshold': 26,
 'target_streak_feat_n': 'low_case_streak',
 'initial_streaks': True,
 'low_value_range': 2}

In [16]:
zero_one_targets = value_streaks(data=df_labels_clean, column=TARGET, value=range(2),
                             run_threshold=1)
print("Zero and one consecutive value streaks for target data (total dengue cases).")
zero_one_targets

Zero and one consecutive value streaks for target data (total dengue cases).


,first_pos,last_pos,streak_len
0,930,1004,75
1,1073,1081,9
2,1089,1094,6
3,1339,1344,6
4,1196,1200,5
5,1238,1240,3
6,1098,1100,3
7,1264,1266,3
8,1335,1337,3
9,1388,1389,2


In [17]:
# def _get_cumulative_streaks(group: pd.Series,
#                             low_value_range:int,
#                            filter_initial_streaks: bool=False):
#     """
#     Compute low-value streak lengths within a 1D series, optionally filtering to initial streak only.

#     :param group: pandas Series representing a single grouped sequence (e.g., one city).
#     :param low_value_range: Upper bound (exclusive) for values considered "low".
#     :param filter_initial_streaks: If True, keep only initial low-value streak when it starts at first
#                                    position (all others 0). If False, return complete streak lengths.
#                                    Default is False.
#     :return: pandas Series (same index/shape as `group`) with cumulative low-value streak lengths,
#              filtered according to `filter_initial_streaks`.
#     """
    
#     mask_lows = group.isin(range(low_value_range))
#     streak_groups = (mask_lows != mask_lows.shift(fill_value=False)).cumsum()
#     low_count_streaks = mask_lows.groupby(by=streak_groups).cumsum()

#     if not filter_initial_streaks:
#         return low_count_streaks

#     else:
#         if low_count_streaks.iloc[0] == 1:
#             initial_low_streaks = low_count_streaks.where(streak_groups == 1, 0)
#         else:
#             initial_low_streaks = pd.Series(0, index=low_count_streaks.index)
#         return initial_low_streaks

In [18]:
# def low_value_targets(X: pd.DataFrame, y: pd.DataFrame,
#                       target_feature: str | None=None,
#                       group_feature:str | None=None,
#                       new_feat_name:str | None=None,
#                       initial_streaks_only:bool | None=None,
#                       min_initial_streak_len:int | None=None,
#                       low_value_range:int | None=None
#                      ) -> pd.DataFrame:
#     """
#     Generate low-value streak features for ML pipelines. Creates continuous streak length feature
#     (`low_case_streak`) and optional boolean initial-streak indicator with minimum length filtering.
    
#     Continuous streak preserves magnitude info for model gradients. Boolean initial streak supports
#     minimum length thresholding.

#     :param X: Input features DataFrame.
#     :param y: Target DataFrame (same index as X).
#     :param target_feature: Target column name. Uses config default if None.
#     :param group_feature: Grouping column (e.g., 'city'). None processes entire series.
#                             Defaults to config settings.
#     :param new_feat_name: Output base name (e.g., 'low_case_streak'). None returns X unchanged.
#                             Defaults to config settings.
#     :param initial_streaks_only: If True, adds thresholded boolean `initial_{new_feat_name}`.
#                             Defaults to config settings.
#     :param min_initial_streak_len: Min length threshold. Applies **only** to boolean `initial_*` feature.
#                             Defaults to config settings.
#     :param low_value_range: Values < this are "low" (exclusive upper bound).
#                             Defaults to config settings.
#     :return: X with `low_case_streak` (continuous) ± `initial_low_case_streak` (boolean).
#     """
    
#     assert all(X.index == y.index), "Indices for 'X' and 'y' must be aligned." 
    
#     config_values = cnfg.preprocess
    
#     target_feature = target_feature or config_values.feature_groups["target"]
#     group_feature = group_feature or config_values.feature_groups["city"]
#     min_initial_streak_len = min_initial_streak_len or (config_values.
#         low_value_streak_features["target_streak_len_threshold"])
#     new_feat_name = new_feat_name or (config_values.
#         low_value_streak_features.get("target_streak_feat_n"))
#     low_value_range = low_value_range or (config_values.
#         low_value_streak_features["low_value_range"])

#     if initial_streaks_only is None:
#         initial_streaks_only = config_values.low_value_streak_features.get("initial_streaks"
#                                                                           ) or False
#     if new_feat_name is None:
#         return X
        
#     X_low_streaks = X.copy()

#     if group_feature is not None:
#         low_count_streaks = y.groupby(by=group_feature)[target_feature].transform(
#             lambda x: _get_cumulative_streaks(x, low_value_range))
#     else:
#         low_count_streaks = _get_cumulative_streaks(y[target_feature], low_value_range)
        
#     temp_output_dict = {new_feat_name: low_count_streaks}

#     if initial_streaks_only:
#         if group_feature is not None:
#             initial_streaks = y.groupby(by=group_feature)[target_feature].transform(
#                 lambda x: _get_cumulative_streaks(x, low_value_range, initial_streaks_only))
#         else:
#             initial_streaks = _get_cumulative_streaks(y[target_feature], low_value_range,
#                                                       initial_streaks_only)
            
#         if min_initial_streak_len:
#             long_streak_mask = initial_streaks >= min_initial_streak_len
#             long_strek_threshpoints = long_streak_mask[
#                 initial_streaks == min_initial_streak_len].index
            
#             for point in long_strek_threshpoints:
#                 start = max(0, point - min_initial_streak_len + 1)
#                 long_streak_mask[start:point] = True
#             initial_streaks = initial_streaks.where(long_streak_mask, 0)
            
#         temp_output_dict[f"initial_{new_feat_name}"] = initial_streaks
            
            
#     for feat_name, streak in temp_output_dict.items():
#         if feat_name.startswith("initial_"):
#             streak = streak.astype(bool).astype(int)
#         X_low_streaks[feat_name] = streak
        
#     return X_low_streaks

In [19]:
df_train_eng = low_value_targets(X=df_train_eng, y=df_labels_clean)
df_train_eng.columns

Index(['city', 'year', 'weekofyear', 'week_start_date', 'precipitation_amt_mm',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'missing_pct_env_feat',
       'missing_station', 'missing_ndvi_s', 'missing_ndvi_n', 'ndvi_north',
       'ndvi_south', 'low_case_streak', 'initial_low_case_streak'],
      dtype='object')

In [20]:
df_train_eng["initial_low_case_streak"].sum()

75

In [21]:
df_train_eng["low_case_streak"].sum()

3029

### Cyclical time features
- [X] Create circular sin/cos features based on `weekofyear`
- [X] drop source time feature `weekofyear`
- [X] keep `week_start_date` for time being for train-test-splits and windowing
- [ ] remove `week_start_date` before modeling

In [22]:
# def circular_time_features(X: pd.DataFrame,
#                            source_feature: str | None = None,
#                            period: int | None = None,
#                            drop_source_feature: bool = True):
#     """
#     Generate cyclical sin/cos features for periodic time variables. Transforms integer week/month/day
#     features into continuous circular encodings that preserve temporal continuity across period boundaries.

#     :param X: Input features DataFrame.
#     :param source_feature: Periodic column name (e.g., 'weekofyear'). Uses config default if None.
#     :param period: Fixed cycle length (e.g., 52 for weeks, 12 for months). Uses column max if None.
#     :param drop_source_feature: If True, drops original source column after encoding.
#     :return: X with `sin_{source_feature}` and `cos_{source_feature}` columns added.
#     """
#     source_feature = (source_feature 
#                       or cnfg.preprocess.feature_groups["week"])
    
#     _check_feature_presence(source_feature, X.columns)
    
#     X_circular = X.copy()
#     divisor = period or X_circular[source_feature].max()
    
#     X_circular[f"sin_{source_feature}"] = np.sin(
#         2 * np.pi * X_circular[source_feature] / divisor)
#     X_circular[f"cos_{source_feature}"] = np.cos(
#         2 * np.pi * X_circular[source_feature] / divisor)

#     if drop_source_feature:
#         X_circular.drop(columns=[source_feature], inplace=True)

#     return X_circular

In [23]:
df_train_eng = circular_time_features(df_train_eng)
[feature for feature in df_train_eng.columns if "week" in feature]

['week_start_date', 'sin_weekofyear', 'cos_weekofyear']

In [24]:
df_train_eng.shape[1]

22

- Post-initial-engineering (excluding lags/rollings) Variance Inflation Factor ratios: VIF 3.

In [25]:
print("Post-initial-engineering (excluding lags/rollings) Variance Inflation Factor ratios: VIF 3.")
temp_vif_pre = df_train_eng.groupby(by=CITYGROUP_FEAT).apply(lambda group: top_vif(data=group), include_groups=False)
temp_vif_pre =  temp_vif_pre.loc["iq"].join(temp_vif_pre.loc["sj"], lsuffix="_iq", rsuffix="_sj", how="inner")
temp_vif_pre["vif_total"] = top_vif(data=df_train_eng).values
temp_vif_pre.sort_values(by="vif_total", ascending=False, na_position="first")

Post-initial-engineering (excluding lags/rollings) Variance Inflation Factor ratios: VIF 3.


/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,vif_iq,vif_sj,vif_total
missing_ndvi_n,inf,13.927079,14.723138
missing_ndvi_s,inf,10.325813,10.209664
missing_pct_env_feat,7.697195,29.656409,8.888056
missing_station,7.221251,1.814554,7.535969
reanalysis_tdtr_k,6.571043,1.833562,6.770829
reanalysis_relative_humidity_percent,6.421888,3.115316,6.145424
initial_low_case_streak,4.941090,NaN,5.764552
ndvi_south,4.543327,1.119917,5.580432
ndvi_north,4.308246,1.387938,4.297752
low_case_streak,3.752724,1.051874,4.269631


In [26]:
corr_threshold=0.8
print(f"City-stratified correlations exceeding {corr_threshold}:")
df_train_eng.groupby(
    by=CITYGROUP_FEAT).apply(
        lambda group: top_correlations(
            data=group, 
            corr_threshold=corr_threshold
        ), include_groups=False)

City-stratified correlations exceeding 0.8:


city                                                               
iq    missing_ndvi_s           missing_ndvi_n                          1.000000
      missing_pct_env_feat     missing_station                         0.889715
      ndvi_north               ndvi_south                              0.872014
      initial_low_case_streak  low_case_streak                         0.847191
      reanalysis_tdtr_k        reanalysis_relative_humidity_percent   -0.900588
sj    station_min_temp_c       station_avg_temp_c                      0.898016
      station_max_temp_c       station_avg_temp_c                      0.864780
      missing_ndvi_n           missing_pct_env_feat                    0.808460
dtype: float64

**Conclusion**:
- Scores are high for `missing_ndvi_n` and `missing_ndvi_s`. This is expected behaviour as EDA showed simultaneous vegetation satelite data outage periods in `San Juan` while `Iqiotos` sub-set had no NDVI missingnes thus perfect correlation with all 0 values. Keep missingness features and increase acceptable VIF threshold for the project to `15`. Both `LightGBM` and `LSTM` should handle it well.
- [ ] `missing_pct_env_feat` has VIF spike of 29 for `San Juan`. Pairwise correlation analysis indicates parallels with `missing_station`. Both encompass different signals and total VIF is acceptable. So keep both. Hovever,monitor in Walk-forward Cross-Validation.
- [ ] `station_avg_temp_c` has increased in `San Juan` subgroup from ~ 15 to ~ 25. However total remains at ~ 5. So keep, but monitor in Walk-forward Cross-Validation.
- `reanalysis_tdtr_k` has increased to over 10, but slightly - still acceptable, so keep
- [ ] `initial_low_case_streak` behaves erratically as in `San Juan` dataset its value is constant = 0 for all rows. Consider removing if Walk-forward Cross-Validation `Test C:` vs `Test B` does not show sufficient improvememnt.
- grouping `ndvi` into pairwise south and north (experiment visible here) or east and west quadrants (experiment not visible above) did not reduce their colineariy. Aggregating into single `ndvi` vegetation feature risks loosing too much signal for the model.
    - [ ] Exclude aggregating vegetation from base pipeline
    - [ ] compare feature Walk-forward Cross-Validation performance for `Test A` (no agregation) and `Test E` (pairwise east/west aggregation)

### (Optional) Lags / rolling stats
- If feature space limited (as is the case), prioritize rollings over multiple lags
- [X] Historical total_cases lags/rollings (t-1 to t-4, or 4-week rolling mean)
- [X] Cumulative effects (rainfall, precipitation, ie `precipitation_amt_mm`) - rolling sum 4 weeks
- [X] Environmental station features 4 week rolling mean for avg:
    - [X] temperature (`station_avg_temp_c`)
    - [X] humidity (`reanalysis_relative_humidity_percent`)
- [ ] Add Walk-forward CV to feature selection notebook in otder to validate performance of new features

In [27]:
df_train_eng.columns

Index(['city', 'year', 'week_start_date', 'precipitation_amt_mm',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'missing_pct_env_feat',
       'missing_station', 'missing_ndvi_s', 'missing_ndvi_n', 'ndvi_north',
       'ndvi_south', 'low_case_streak', 'initial_low_case_streak',
       'sin_weekofyear', 'cos_weekofyear'],
      dtype='object')

In [28]:
cnfg.preprocess.dynamic_temporal_features

{'target': {'source_feature': 'total_cases',
  'task': 'lag',
  'backward_steps': [1, 4]},
 'precipitation': {'source_feature': 'precipitation_amt_mm',
  'task': 'sum',
  'backward_steps': 4},
 'temperature': {'source_feature': 'station_avg_temp_c',
  'task': 'mean',
  'backward_steps': 4},
 'humidity': {'source_feature': 'reanalysis_relative_humidity_percent',
  'task': 'mean',
  'backward_steps': 4}}

In [29]:
# def _rolling_aggregations(groups, task, step):
#     """Handle rolling aggregation (mean, sum, etc.) for the given step."""
#     return groups.transform(lambda group: group.rolling(
#         window=step + 1, min_periods=1, closed="left").agg(task))

# def _lags(groups, step):
#     """Handle lag features for the given step."""
#     return groups.transform(lambda group: group.shift(step))
    
    
# def dynamic_temporal_features(X: pd.DataFrame,
#                              y: pd.Series | None = None,
#                              target_feature: str | None=None,
#                               group_feature: str | None=None,
#                              source_feature: str | None=None,
#                               new_feature: str | None=None,
#                              task: str | None=None,
#                              backward_steps: int | None=None):

#     """
#     Generate dynamic temporal features (e.g., rolling aggregates (mean, sum, etc.),
#     lag features).
#     Remove NaNs rows created in the proces. 
#     WARNING: If input X contains NaN rows, those will be removed.
    
#     :param X: Input pandas DataFrame containing features.
#     :param y: Optional pandas Series of target values.
#     :param target_feature: The name of the target feature in `y` to include for lag creation. 
#                             If None, uses default from configuration.
#     :param group_feature: The name of the grouping feature (e.g., city, region) for temporal 
#                             aggregation. Defaults to 'city' from config.
#     :param source_feature: The name of the feature to transform (e.g., 'total_cases',
#                             'precipitation_amt_mm'). If None, uses default from configuration.
#     :param new_feature: The name of the new feature column to be created. 
#                         If None, uses default from configuration.
#     :param task: The type of transformation to apply. Options are 'lag', 'sum', or 'mean'. 
#                     If None, uses default from configuration.
#     :param backward_steps: List or a single integer of steps to look back for lag or rolling 
#                             aggregation. Example: [1, 4] means lags/rolling aggregation for 1 
#                             and 4 steps. If None, uses default from configuration. 
#     :return: DataFrame with dynamic temporal features added. If `y` is provided, returns 
#                 2 dataframes X with new fetures and y both filtered to remove 
#                 any rows with NaNs.
#     """
    
#     target_feature = target_feature or cnfg.preprocess.feature_groups.get("target")
#     group_feature = group_feature or cnfg.preprocess.feature_groups.get("ciy")

#     X_temporal = X.copy()
    
#     feature_params = dict()
#     if all((param is None for param in (
#         source_feature, new_feature, task, backward_steps))):
#         feature_params = cnfg.preprocess.dynamic_temporal_features
#     else:
#         feature_params = {source_feature:
#                           {"source_feature": source_feature,
#                            "task": task,
#                            "backward_steps": backward_steps}}

#     if group_feature is None:
#         group_feature = pd.Series(data=np.ones(len(X_temporal), dtype=bool),
#                                   index=X_temporal.index)
        
#     switch=False
    
#     if (any(target_feature == v["source_feature"] for k,v in feature_params.items()) 
#         and y is not None):
#         X_temporal[target_feature] = y[target_feature]
#         switch = True
     
#     for param, rules in feature_params.items():
#         if not isinstance(rules["backward_steps"], list):
#             rules["backward_steps"] = [rules["backward_steps"]]

#         if rules["task"] not in ["lag", "sum", "mean", "max", "min"]:
#             logging.warning(f"Wrong task type provided: {rules['task']}.")
            
#         _check_feature_presence(rules["source_feature"], X_temporal.columns)
#         groups = X_temporal.groupby(by=group_feature)[rules["source_feature"]]     
#         for step in rules["backward_steps"]:
#             if rules["task"] != "lag":
#                 X_temporal[
#                     f'{param}_rolling_{rules["task"]}_{step}'
#                     ] = _rolling_aggregations(groups, rules["task"], step)
#             else:
#                 X_temporal[
#                     f'{param}_{rules["task"]}_{step}'
#                     ] = _lags(groups, step)
    
#     if switch:
#         X_temporal.drop(columns=[target_feature], inplace=True)
        
#     mask = X_temporal.notna().all(axis=1)
#     X_temporal = X_temporal[mask]

#     if y is not None:
#         return X_temporal, y.copy()[mask]
        
#     return X_temporal

In [30]:
df_train_eng, df_labels_eng = dynamic_temporal_features(df_train_eng, df_labels_clean)

In [31]:
new_feats = [feature for feature in df_train_eng.columns if feature.startswith(tuple(cnfg.preprocess.dynamic_temporal_features.keys()))]
df_train_eng[new_feats].sample(3)

,precipitation_amt_mm,target_rolling_lag_1,target_rolling_lag_4,precipitation_sum_4,temperature_mean_4,humidity_mean_4
988,5.14,0.0,0.0,174.8075,27.020667,82.454571
505,24.19,9.0,23.0,126.9200,24.658514,74.460046
391,50.55,47.0,70.0,201.8700,27.705714,79.562571


In [32]:
print("Check if X an y indexes align:")
all(df_labels_eng.index == df_train_eng.index)

Check if X an y indexes align:


True

- Post feature engineering (including lags/rollings) Variance Inflation Factor ratios: VIF 4.

In [33]:
print("Post feature engineering (including lags/rollings) Variance Inflation Factor ratios: VIF 4.")
temp_vif_pre = df_train_eng.groupby(by=CITYGROUP_FEAT).apply(lambda group: top_vif(data=group), include_groups=False)
temp_vif_pre =  temp_vif_pre.loc["iq"].join(temp_vif_pre.loc["sj"], lsuffix="_iq", rsuffix="_sj", how="inner")
temp_vif_pre["vif_total"] = top_vif(data=df_train_eng).values
temp_vif_pre.sort_values(by="vif_total", ascending=False, na_position="first")

Post feature engineering (including lags/rollings) Variance Inflation Factor ratios: VIF 4.


/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,vif_iq,vif_sj,vif_total
missing_ndvi_s,inf,11.175960,14.816600
missing_ndvi_n,inf,13.949696,10.618118
missing_pct_env_feat,7.779426,29.729477,10.553650
missing_station,7.293415,1.827712,8.156276
reanalysis_relative_humidity_percent,6.813269,3.364903,7.034363
reanalysis_tdtr_k,6.686965,1.852951,6.249533
initial_low_case_streak,5.579551,NaN,5.789321
ndvi_south,4.568297,1.142279,5.640069
ndvi_north,4.327275,1.414127,5.414341
low_case_streak,3.910723,1.057828,5.296480


In [34]:
corr_threshold=0.8
print(f"City-stratified correlations exceeding {corr_threshold}:")
df_train_eng.groupby(
    by=CITYGROUP_FEAT).apply(
        lambda group: top_correlations(
            data=group, 
            corr_threshold=corr_threshold
        ), include_groups=False)

City-stratified correlations exceeding 0.8:


city                                                               
iq    missing_ndvi_n                        missing_ndvi_s             1.000000
      missing_pct_env_feat                  missing_station            0.889715
      ndvi_north                            ndvi_south                 0.872014
      low_case_streak                       initial_low_case_streak    0.847191
      reanalysis_relative_humidity_percent  reanalysis_tdtr_k         -0.900588
sj    station_min_temp_c                    station_avg_temp_c         0.898044
      target_rolling_lag_4                  target_rolling_lag_1       0.868119
      station_avg_temp_c                    station_max_temp_c         0.864934
                                            temperature_mean_4         0.843385
      missing_ndvi_n                        missing_pct_env_feat       0.808289
      station_min_temp_c                    temperature_mean_4         0.805664
      temperature_mean_4                    sin_week

In [35]:
df_train_eng.shape[1]

27

In [36]:
df_train_eng.columns

Index(['city', 'year', 'week_start_date', 'precipitation_amt_mm',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'missing_pct_env_feat',
       'missing_station', 'missing_ndvi_s', 'missing_ndvi_n', 'ndvi_north',
       'ndvi_south', 'low_case_streak', 'initial_low_case_streak',
       'sin_weekofyear', 'cos_weekofyear', 'target_rolling_lag_1',
       'target_rolling_lag_4', 'precipitation_sum_4', 'temperature_mean_4',
       'humidity_mean_4'],
      dtype='object')

**Conclusion**:
- overall VIF increases for rolling stat source features are minimal - below 10%. Thu keep both source and engineered features.
- [ ] `station_avg_temp_c` in `San Juan`  had issues before and incresed modrrately from previous ~25 to ~ 27. However total ramains at ~ 5. So keep, but monitor in Walk-forward Cross-Validation.
- `reanalysis_relative_humidity_percent` has increased to over 10, but slightly - still acceptable, so keep
- New features `humidity_rolling_mean_4` and `temperature_rolling_mean_4` have lower VIFs than their sources
- Target lags `target_lag_` 1/4 have low multicolinearity.

### Engineering pipeline
- [X] Load clean data
- [ ] run engineering steps in sequence
- [X] save engineered data

In [37]:
# def pipe_engineer(X: pd.DataFrame | None=None,
#                   y: pd.DataFrame | None=None,
#                   nan_mask: pd.DataFrame | None=None,
#                   manual_dirs: Dict[str, Path] | None=None,
#                   manual_files: Dict[str, Path] | None=None,
#                   datetime_col: str=None,
#                   overwrite_files: bool=False) -> Dict[str, Any]:
#     """
#     Feature engineering pipeline for dengue case prediction.
#     Combines missingness indicators, feature reduction, outlier handling, 
#     cyclical encoding, and dynamic temporal lags/rolling statistics.
    
#     :param X: Input features DataFrame. Default None uses config.yaml data.files.
#     :param y: Input targets DataFrame. Default None uses config.yaml data.files.
#     :param nan_mask: NaN mask DataFrame tracking imputation locations. 
#         Default None uses config.yaml data.dirs.
#     :param manual_dirs: Dict of directory paths to override config.yaml dirs. 
#         Default None uses config.yaml data.dirs.
#     :param manual_files: Dict of filenames to override config.yaml files. 
#        Default None uses config.yaml data.files.
#     :param datetime_col: Name of datetime column for parsing. Default None uses
#         config.yaml preprocess.feature_groups["datetime"].
#     :param overwrite_files: If True, overwrites existing engineered feature files.
#         Default False.
    
#     :return: Dict containing:
#         - 'X_eng_save_path': Path where engineered features saved
#         - 'X_eng_data': Engineered features DataFrame
#                         (missingness, temporal lags/rolls, cyclical)
#         - 'y_eng_save_path': Path where engineered targets saved  
#         - 'y_eng_data': Engineered targets DataFrame (Remowed first rows due to
#                         NaNs created by lag/rolling stat feature engineering)
#     """
#     dirs = manual_dirs or cnfg.data.dirs
#     filenames = manual_files or cnfg.data.files
#     datetime_col = datetime_col or cnfg.preprocess.feature_groups["datetime"]

#     file_loaders = {
#         'X': lambda: load_file(path=dirs["intermediate"] / 
#                                filenames["features_clean"],
#                                datetime_col=datetime_col),
#         'y': lambda: load_file(path=dirs["intermediate"] /
#                                filenames["labels_clean"],
#                                datetime_col=datetime_col),
#         "nan_mask": lambda: load_file(path=dirs["intermediate"] /
#                                       filenames["nan_mask"],
#                                       datetime_col=datetime_col)
#     }

#     X_eng = X.copy() if X is not None else file_loaders['X']()
#     y_eng = y.copy() if y is not None else file_loaders['y']()
#     nan_mask = (nan_mask.copy() if nan_mask is not None 
#                 else file_loaders["nan_mask"]())

#     X_eng = add_missingness_features(X=X_eng, nan_mask=nan_mask)
#     X_eng = reduce_features(X=X_eng)
#     X_eng = low_value_targets(X=X_eng, y=y_eng)
#     X_eng = circular_time_features(X=X_eng)
#     X_eng, y_eng = dynamic_temporal_features(X=X_eng, y=y_eng)
      
#     X_save_path = save_file(df=X_eng, path=dirs["intermediate"] / 
#                             filenames["features_eng"],
#                             overwrite=overwrite_files)
#     y_save_path = save_file(df=y_eng, path=dirs["intermediate"] / 
#                             filenames["labels_eng"],
#                             overwrite=overwrite_files)
    
#     return {"X_eng_save_path": X_save_path,
#             "X_eng_data": X_eng,
#             "y_eng_save_path": y_save_path,
#             "y_eng_data": y_eng,         
#            }

In [38]:
%%time
pipe_test = pipe_engineer(X=df_train_clean, y=df_labels_clean, nan_mask=df_nan_mask, overwrite_files=True)

INFO:root:Saved (1442, 27) shaped data as dengue_features_eng.parquet
INFO:root:Saved (1442, 4) shaped data as dengue_labels_eng.parquet


CPU times: user 104 ms, sys: 2.02 ms, total: 106 ms
Wall time: 61.5 ms


In [39]:
%%time
pipe_test = pipe_engineer(overwrite_files=True)

INFO:root:Path file present, overwriting.
INFO:root:Saved (1442, 34) shaped data as dengue_features_eng.parquet
INFO:root:Path file present, overwriting.
INFO:root:Saved (1442, 4) shaped data as dengue_labels_eng.parquet


CPU times: user 95.3 ms, sys: 2.96 ms, total: 98.3 ms
Wall time: 96.7 ms


In [40]:
pipe_test.keys()

dict_keys(['X_eng_save_path', 'X_eng_data', 'y_eng_save_path', 'y_eng_data'])

In [41]:
all(pipe_test["X_eng_data"].index == df_labels_eng.index)

True

In [42]:
all(pipe_test["y_eng_data"].index == df_train_eng.index)

True